In [6]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
import nltk
from nltk import sent_tokenize
nltk.download('punkt')
import warnings
import ast
warnings.filterwarnings(action='ignore')
import json 
from requests import get
import datetime
from datetime import date
from tqdm import tqdm
import pickle
import boto3
from collections import Counter

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
df =pd.read_csv('data/Tags.csv')
tags_order = [i[0] for i in Counter([i.lower() for i in df['Label'].dropna()]).most_common()][::-1]
del df['count']
tag_map = df.set_index('tags').to_dict('index')
df = pd.read_pickle('UIdata/data_tagging.pkl')

In [68]:
for i in df.tags:
    if i == 'Iidian heritage sites':
        print(i)

In [10]:
def get_valid_labels(tag_list):
    df['tags'][0]

['Current Affairs 2013',
 'February 2013',
 'IBPS',
 'January 2013',
 'Supreme court']

In [65]:
tag_list = df['tags'].values[-100]

In [66]:
tag_list

['Archaeological Survey of India (ASI)',
 'heritage sites',
 'Indian heritage Sites',
 'Museum Grant Scheme',
 'Sites Identified for ‘National importance’ tag',
 'Sites of National Importance']

In [36]:
labels_  = []
for i in tag_list:
    i = i.lower()
    if i in tag_map.keys():
        labels_.append(tag_map[i])
        

In [37]:
labels_

[]

In [16]:
tags_order

['agriculture ',
 'history',
 'agriculture',
 'geography',
 'society ',
 'health',
 'culture',
 'defence',
 'governance',
 'environment',
 'polity',
 'science ',
 'economy',
 'international_relations']

In [7]:
df = pd.read_pickle('UIdata/data_tagging.pkl')

In [10]:
__TableName__ = 'upsc_training_data'
client  = boto3.client('dynamodb',region_name = 'ap-south-1')
DB  = boto3.resource('dynamodb',region_name = 'ap-south-1')
table = DB.Table(__TableName__)

In [52]:
tags = []
headings = []
c = 0
for i in tqdm(range(27000)):
    try:
        item = table.get_item(Key={'TrainId':i})
        tags.append(item['Item']['tags'])
        heading.append(item['Item']['heading'])
    except:
        c = c+1

with open('data/tags.pkl', 'wb') as f:
    pickle.dump(tags, f)
with open('data/headings.pkl', 'wb') as f:
    pickle.dump(headings, f)

  1%|          | 286/27000 [00:14<26:55, 16.53it/s] 

In [65]:
import pickle
import pandas as pd
with open('data/tags.pkl', 'rb') as handle:
    tags = pickle.load(handle)
with open('data/headings.pkl', 'rb') as handle:
    headings = pickle.load(handle)
df = pd.DataFrame()
df['headings'] = headings
df['tags'] = tags
df['TrainId'] = np.arange(len(df))
df.to_pickle('UIdata/data_tagging.pkl')

In [4]:
df = pd.read_pickle('UIdata/data_tagging.pkl')

In [5]:
df

,headings,tags,TrainId
0,JJB rejects bone ossification test plea,"[Current Affairs 2013, February 2013, IBPS, Ja...",0
1,India's first ‘ebiz’ government-to-business po...,"[Current Affairs 2013, IBPS, January 2013]",1
2,"At 105, Ustad Abdul Rashid Khan is the oldest ...","[Current Affairs 2013, IBPS, January 2013]",2
3,Singer S. Janaki declines Padma Bhushan,"[Current Affairs 2013, Entertainment, January ...",3
4,"India, Bangladesh ink extradition and visa pacts","[Current Affairs 2013, India-Bangladesh, India...",4
...,...,...,...
26995,Sample Registration System (SRS) Statistical R...,"[Infant mortality, Infant Mortality Rate, Neon...",26995
26996,Current Affairs- Today’s Headlines: September ...,"[2022, Current Affairs, Current Affairs: News ...",26996
26997,Breakthrough Prizes 2023- Winners,"[Breakthrough Prize, Breakthrough Prize in Fun...",26997
26998,Committee for pharma marketing practices,"[CBDT, Department of Pharmaceuticals, Pharmace...",26998


In [61]:
labels = ['Environment','History','Culture','Geography','International Relations/Issues',
'Polity','Governance','Health','Society','Economy','Science&Technology','Defence','Agriculture']

In [1]:
def get_response_soup(url):
    response = get(url,verify=False, timeout=30)
    cleaned_response = response.text.replace('\x00', '')
    soup = BeautifulSoup(cleaned_response)
    return soup

In [9]:
import datetime
import pandas as pd
from requests import get
import datetime
from datetime import date
start = datetime.datetime.strptime("02-01-2018", "%d-%m-%Y")
end = datetime.datetime.strptime("15-01-2018", "%d-%m-%Y")
date_generated = pd.date_range(start, end)
tags = []
for i in tqdm(range(len(date_generated))):
    date = str(date_generated[i]).split(' ')[0]
    link = 'https://www.iasparliament.com/current-affairs/archives/'+date.replace('-','/')
    soup = get_response_soup(link)
    for i in soup.find_all(name = 'a',attrs = {'class':'label label-default label-default-cntrl tags'}):
        tags.append(i.text) 
        


from collections import Counter
tags_ = [i for i in tags if '-' in i]
dict_ = dict(Counter(tags_))
df_tags = pd.DataFrame()
df_tags['tags'] = tags_

100%|██████████| 14/14 [00:01<00:00,  9.87it/s]


In [12]:
i.text

'G.S II - Bilateral/International Relations'

In [13]:
link

'https://www.iasparliament.com/current-affairs/archives/2018/01/15'

In [121]:
import pickle
import pandas as pd
import itertools
from collections import Counter
with open('data/tags.pkl', 'rb') as handle:
    tags_list = pickle.load(handle)
''' 
tags = []
for i in tags_list:
    tags = tags + i
'''
tags = list(itertools.chain.from_iterable(tags_list))
months_str = ['january', 'february', 'march',
        'april', 'may', 'june', 'july', 'august',
        'september', 'october', 'november', 'december','current affairs','ibps']
tags = [i.lower() for i in tags]
for i in tags:
    for j in months_str:
        if j in i:
            try: 
                tags.remove(i)
            except:
                a = 1
dict_ = Counter(tags)
df = pd.DataFrame()
df['keys_'] = dict_.keys()
df['count'] = [dict_[i] for i in dict_.keys()]
df.to_csv('data/keys_upsc.csv')

In [124]:
labels = ['Environment','History','Culture','Geography','International Relations',
'Polity','Governance Polity','Health','Society','Economy','Science&Technology','Defence','Agriculture']
labels = [i.lower() for i in labels]

In [125]:
labels

['environment',
 'history',
 'culture',
 'geography',
 'international relations/issues',
 'polity',
 'governance polity',
 'health',
 'society',
 'economy',
 'science&technology',
 'defence',
 'agriculture']